In [1]:
import tushare as ts
print(f"tushare version:{ts.__version__}")
token = "8ef5ec61cdd848715c57c11d58dd71da1271f76b2420d2bac8aef123"
import pandas as pd
import numpy  as np
import datetime as dt
import os

tushare version:1.2.54


In [1]:
#主力合约指数

contracts = \
[
'T.CFX', #十债加权-CFFEX10年期国债期货
'TF.CFX', #五债加权-CFFEX5年期国债期货
'AU.SHF', #沪金-黄金主力
'AG.SHF', #沪银-白银主力
'CU.SHF', #沪铜-沪铜主力
'NI.SHF', #沪镍-沪镍主力
'AL.SHF', #沪铝-沪铝主力
'RB.SHF', #螺纹-螺纹钢主力
'I.DCE', #铁矿-铁矿石主力
'J.DCE', #焦炭-焦炭主力
'JM.DCE', #焦煤-焦煤主力
'ZC.ZCE', #郑煤-动力煤主力
'RU.SHF', #橡胶-橡胶主力
'TA.ZCE', #PTA-PTA主力
'MA.ZCE', #郑醇-甲醇主力
'M.DCE', #豆粕-豆粕主力
'P.DCE', #棕榈油-棕榈油主力
'CF.ZCE', #郑棉-棉花主力
'SR.ZCE', #白糖-白糖主力
'C.DCE', #玉米-玉米主力
'SP.SHF', #纸浆-纸浆主力
'CJ.ZCE', #红枣-红枣主力
'ZN.SHF', #沪锌-沪锌主力
'PB.SHF', #沪铅-沪铅主力
'SN.SHF', #沪锡-沪锡主力
'SS.SHF', #SS-不锈钢主力
'HC.SHF', #热卷-热轧卷板主力
'FG.ZCE', #玻璃-玻璃主力
'EB.DCE', #EB-苯乙烯主力
'UR.ZCE', #尿素-尿素主力
'EG.DCE', #EG-乙二醇主力
'SA.ZCE', #纯碱-纯碱主力
#NR-20号胶(missing)
'V.DCE', #PVC-PVC主力
'PP.DCE', #PP-聚丙烯主力
'SM.ZCE', #锰硅-锰硅主力
'SF.ZCE', #硅铁-硅铁主力
'A.DCE', #豆一主力
'B.DCE', #豆二主力
'Y.DCE', #豆油-豆油主力
'OI.ZCE', #郑油-菜油主力
'RM.ZCE', #菜粕-菜油主力
'FU.SHF', #燃油-燃油主力
'PG.DCE', #LPG-LPG主力
'BU.SHF', #沥青-沥青主力
#原油-上期原油 (missing)
'AP.ZCE', #苹果-苹果主力
'L.DCE', #塑料-塑料主力
'CS.DCE', #淀粉-玉米淀粉主力
]

In [52]:
class Tushare(object):
    def __init__(self, contracts):
        self.pro = ts.pro_api(token)
        self.contracts = contracts
        
        self.daily_main_contracts = []
        
    # save futures data to csv
    def df_xlsx(self):
        if not os.path.exists('futures_main_contracts'):
                os.makedirs('futures_main_contracts')
        
        print("# start fetching futures' main contracts data")
        for contract in self.contracts:
            # 获取主力合约的数据
            data = self.pro.fut_daily(ts_code = contract, fields='ts_code,trade_date,open,high,low,close,settle,vol,oi').set_index('trade_date')
            data.columns = ['ts_code', 'open', 'high', 'low', 'close', 'settle', 'volume', 'open_interest']
            
            # 获取主力合约的中文商品名字
            exchange = contract.split('.')[1]
            exchanges = {'SHF':'SHFE', 'CFX':'CFFEX', 'ZCE':'CZCE', 'DCE':'DCE'}
            name = self.pro.fut_basic(exchange=exchanges[exchange], fut_type='2')
            name = name[name['ts_code']==contract]['name']
            data['name'] = name.iloc[0]
            
            # 获取每天对照主力合约的具体合约的名字
            specific_main_contract =self.pro.fut_mapping(ts_code=contract,fields='trade_date,mapping_ts_code').set_index('trade_date')
            data = data.join(specific_main_contract,how='outer')
            self.daily_main_contracts.append(specific_main_contract['mapping_ts_code'].iloc[0])
            
            # dataframe to excel file
            columns = ['name','ts_code', 'mapping_ts_code','open', 'high', 'low', 'close', 'settle', 'volume', 'open_interest']
            data = data[columns]
            data.sort_index(ascending=False, inplace=True)
#             data.to_excel('futures_main_contracts/{}.xlsx'.format(name.iloc[0]))
            print("-----Loaded {}.xlsx".format(name.iloc[0]))
        
        print("\nToday's main contracts:", self.daily_main_contracts)
        
        
    # 计算今天的主力合约的交易价格（一手）
    def cost(self, num_contract=1):
        costs = {}
        main_df = pd.DataFrame()
        
        if not self.daily_main_contracts:
            for contract in self.contracts:
                specific_main_contract = self.pro.fut_mapping(ts_code=contract,fields='trade_date,mapping_ts_code').set_index('trade_date')
                self.daily_main_contracts.append(specific_main_contract['mapping_ts_code'].iloc[0])
        
        print("# start calculate buy costs of futures' main contracts")
        for contract in self.daily_main_contracts:
            
            # 获取每个contract的交易单位（合约乘数），用来计算保证金
            exchange = contract.split('.')[1]
            exchanges = {'SHF':'SHFE', 'CFX':'CFFEX', 'ZCE':'CZCE', 'DCE':'DCE'}
            
#             detail = self.pro.fut_basic(exchange=exchanges[exchange], trade_date=str(dt.date.today()).replace('-',''))
            detail = self.pro.fut_basic(exchange=exchanges[exchange], trade_date="20200629")
            multiplier = detail[detail['ts_code']==contract]['per_unit'].iloc[0]
            
            # 获取交易手续费，交易保证金
#             data = self.pro.fut_settle(ts_code=contract,trade_date = str(dt.date.today()).replace('-',''))
            data = self.pro.fut_settle(ts_code=contract,trade_date = "20200629")
            data['multiplier'] = multiplier
#             print(data)
            data['buy_cost'] = (data['trading_fee'] + data['settle'] * (data['trading_fee_rate']/100) * data['multiplier'] + data['settle'] * data['multiplier'] * data['long_margin_rate']) * num_contract
            data.set_index('ts_code',inplace=True)
            data = data[['trade_date','settle','trading_fee_rate','trading_fee','long_margin_rate','multiplier','buy_cost']]
            
            try: 
                buy_cost = data['buy_cost'].iloc[0]
            except:
                buy_cost = np.nan
                
            main_df = pd.concat([main_df,data],axis=0,sort=False)
            print('{}: {}'.format(contract,buy_cost))
            
        # 生成csv 文件
        main_df.to_csv('{}_buy_cost.csv'.format(str(dt.date.today()).replace('-','')))
        return main_df
            

In [36]:
# 计算成本时，用当日的价格dataframe中的close来计算成本，而不用fut_settle函数中的settle，原因是当日收盘前若想查看当时的一个contract
# 的成本，由于还未收盘，因此没有settle数据，无法查看。
pro = ts.pro_api(token)

In [51]:
pro.fut_settle(ts_code="AU2012.SHF",trade_date = "20200628")

,ts_code,trade_date,settle,trading_fee_rate,trading_fee,delivery_fee,b_hedging_margin_rate,s_hedging_margin_rate,long_margin_rate,short_margin_rate


In [24]:
# 生成截止到当日的所有主力合约数据，导出Excel表格
TS = Tushare(contracts)
TS.df_xlsx()

# start fetching futures' main contracts data
-----Loaded CFFEX10年期国债期货.xlsx
-----Loaded CFFEX5年期国债期货.xlsx
-----Loaded 黄金主力.xlsx
-----Loaded 白银主力.xlsx
-----Loaded 沪铜主力.xlsx
-----Loaded 沪镍主力.xlsx
-----Loaded 沪铝主力.xlsx
-----Loaded 螺纹钢主力.xlsx
-----Loaded 铁矿石主力.xlsx
-----Loaded 焦炭主力.xlsx
-----Loaded 焦煤主力.xlsx
-----Loaded 动力煤主力.xlsx
-----Loaded 橡胶主力.xlsx
-----Loaded PTA主力.xlsx
-----Loaded 甲醇主力.xlsx
-----Loaded 豆粕主力.xlsx
-----Loaded 棕榈油主力.xlsx
-----Loaded 棉花主力.xlsx
-----Loaded 白糖主力.xlsx
-----Loaded 玉米主力.xlsx
-----Loaded 纸浆主力.xlsx
-----Loaded 红枣主力.xlsx
-----Loaded 沪锌主力.xlsx
-----Loaded 沪铅主力.xlsx
-----Loaded 沪锡主力.xlsx
-----Loaded 不锈钢主力.xlsx
-----Loaded 热轧卷板主力.xlsx
-----Loaded 玻璃主力.xlsx
-----Loaded 苯乙烯主力.xlsx
-----Loaded 尿素主力.xlsx
-----Loaded 乙二醇主力.xlsx
-----Loaded 纯碱主力.xlsx
-----Loaded PVC主力.xlsx
-----Loaded 聚丙烯主力.xlsx
-----Loaded 锰硅主力.xlsx
-----Loaded 硅铁主力.xlsx
-----Loaded 豆一主力.xlsx
-----Loaded 豆二主力.xlsx
-----Loaded 豆油主力.xlsx
-----Loaded 菜油主力.xlsx
-----Loaded 菜粕主力.xlsx
-----Loaded 燃油主力.xlsx
-

In [53]:
# 生成基于目前的主力合约的一个contract的开仓成本
TS = Tushare(contracts)
TS.cost()

# start calculate buy costs of futures' main contracts
T2009.CFX: nan
TF2009.CFX: nan
AU2012.SHF: 32035.600000000002
AG2012.SHF: nan
CU2008.SHF: 24295.875
NI2008.SHF: nan
AL2008.SHF: 6865.5
RB2010.SHF: 3247.79
I2009.DCE: nan
J2009.DCE: nan
JM2009.DCE: nan
ZC2009.ZCE: nan
RU2009.SHF: 11194.1075
TA2009.ZCE: nan
MA2009.ZCE: nan
M2009.DCE: nan
P2009.DCE: nan
CF2009.ZCE: nan
SR2009.ZCE: nan
C2009.DCE: nan
SP2009.SHF: 3545.2200000000003
CJ2009.ZCE: nan
ZN2008.SHF: 8325.5
PB2008.SHF: 7349.28
SN2008.SHF: nan
SS2008.SHF: nan
HC2010.SHF: 3247.79
FG2009.ZCE: nan
EB2009.DCE: nan
UR2009.ZCE: nan
EG2009.DCE: nan
SA2009.ZCE: nan
V2009.DCE: nan
PP2009.DCE: nan
SM2009.ZCE: nan
SF2010.ZCE: nan
A2009.DCE: nan
B2008.DCE: nan
Y2009.DCE: nan
OI2009.ZCE: nan
RM2009.ZCE: nan
FU2009.SHF: 2481.745
PG2011.DCE: nan
BU2012.SHF: 3898.82
AP2010.ZCE: nan
L2009.DCE: nan
CS2009.DCE: nan


,trade_date,settle,trading_fee_rate,trading_fee,long_margin_rate,multiplier,buy_cost
ts_code,,,,,,,
AU2012.SHF,20200629,400.32,0.000,10.0,0.08,1000.0,32035.6000
CU2008.SHF,20200629,48350.00,0.050,0.0,0.10,5.0,24295.8750
AL2008.SHF,20200629,13725.00,0.000,3.0,0.10,5.0,6865.5000
RB2010.SHF,20200629,3569.00,0.100,0.0,0.09,10.0,3247.7900
RU2009.SHF,20200629,10135.00,0.045,0.0,0.11,10.0,11194.1075
SP2009.SHF,20200629,4404.00,0.050,0.0,0.08,10.0,3545.2200
ZN2008.SHF,20200629,16645.00,0.000,3.0,0.10,5.0,8325.5000
PB2008.SHF,20200629,14640.00,0.040,0.0,0.10,5.0,7349.2800
HC2010.SHF,20200629,3569.00,0.100,0.0,0.09,10.0,3247.7900
